<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Weather-Data-EDA" data-toc-modified-id="Weather-Data-EDA-1">Weather Data EDA</a></span></li><li><span><a href="#Mobility-Data-EDA" data-toc-modified-id="Mobility-Data-EDA-2">Mobility Data EDA</a></span></li><li><span><a href="#Standardize-and-Add-Variables" data-toc-modified-id="Standardize-and-Add-Variables-3">Standardize and Add Variables</a></span></li><li><span><a href="#Linear-Regressions" data-toc-modified-id="Linear-Regressions-4">Linear Regressions</a></span></li><li><span><a href="#Aggregate-Predictors-for-LSTM" data-toc-modified-id="Aggregate-Predictors-for-LSTM-5">Aggregate Predictors for LSTM</a></span></li><li><span><a href="#Run-GRU-and-LSTM-Models" data-toc-modified-id="Run-GRU-and-LSTM-Models-6">Run GRU and LSTM Models</a></span><ul class="toc-item"><li><span><a href="#Discussion-of-RNN-Models" data-toc-modified-id="Discussion-of-RNN-Models-6.1">Discussion of RNN Models</a></span></li></ul></li><li><span><a href="#Aggregate-Data-by-County" data-toc-modified-id="Aggregate-Data-by-County-7">Aggregate Data by County</a></span></li><li><span><a href="#Regression-on-R-proxy-Values-and-Predictors" data-toc-modified-id="Regression-on-R-proxy-Values-and-Predictors-8">Regression on R-proxy Values and Predictors</a></span><ul class="toc-item"><li><span><a href="#Discussion-of-Regressions" data-toc-modified-id="Discussion-of-Regressions-8.1">Discussion of Regressions</a></span></li></ul></li><li><span><a href="#Sources:" data-toc-modified-id="Sources:-9">Sources:</a></span></li></ul></div>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from statsmodels.api import OLS

import plotly.figure_factory as ff
import plotly

from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
import plotly.express as px

from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend

from tensorflow.keras import Model, Sequential
from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import Input, SimpleRNN, Embedding, Dense, TimeDistributed, GRU, \
                          Dropout, Bidirectional, Conv1D, BatchNormalization, LSTM

print(tf.keras.__version__)
print(tf.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My\ Drive/Colab\ Notebooks/data/Covid

In [ ]:
# DATA PRE-PROCESSING
# Pre-processed data saved in external csv for ease of use

# import county level COVID-19 data
df_counties = pd.read_csv('COVID-19-Midas/data/cases/USA/nytimes_covid19_data/20200507_us-counties.csv')
df_counties['date_county'] = df_counties['date'] + ' ' + df_counties['county'] + ' ' + df_counties['state']

# import county level weather data
df_temperature = pd.read_csv('temperature_data.csv')
df_rain = pd.read_csv('rain_data.csv')
df_sun = pd.read_csv('sun_data.csv')

# import county level population density data
df_density = pd.read_csv('county_density.csv').dropna()
df_density['GCT_STUB.display-label'] = df_density['GCT_STUB.display-label'].apply(lambda x: x.replace(' County', ''))
df_density['state_county'] = df_density['GEO.display-label'] + ' ' + df_density['GCT_STUB.display-label']

# import county level mobility data from Google's data reports
df_movement = pd.read_csv('Global_Mobility_Report.csv')
df_movement = df_movement.loc[df_movement['country_region_code'] == 'US']
df_movement = df_movement.dropna().reset_index()
df_movement['sub_region_2'] = df_movement['sub_region_2'].apply(lambda x: x.replace(' County', ''))
df_movement['date_county'] = df_movement['date'] + ' ' + df_movement['sub_region_2'] + ' ' + df_movement['sub_region_1']
date_counties = np.array(df_counties['date_county'])

# match date county data accross mobility and COVID data 
df_movement = df_movement.loc[df_movement['date_county'].apply(lambda x: x in date_counties)]

# merge the matched data
df_merged = df_counties.join(df_movement.set_index('date_county'), how ='right', on='date_county', rsuffix='_other')

# drop redundant columns from the merge
df_merged = df_merged.drop(['sub_region_1', 'sub_region_2', 'date_other', 'index', 'country_region_code'], axis=1).reset_index(drop=True)
df_merged['fips'] = df_merged['fips'].apply(lambda x: int(x))
df_merged['state_county'] = df_merged['state'] + ' ' + df_merged['county']

# match weather data to county by FIP and date
temps = []
rains = []
suns = []
for row in df_merged.itertuples():
    try:
        temps.append(df_temperature.loc[df_temperature['date'] == row.date][str(row.fips)].values[0])
        rains.append(df_rain.loc[df_rain['date'] == row.date][str(row.fips)].values[0])
        suns.append(df_sun.loc[df_sun['date'] == row.date][str(row.fips)].values[0])
    except:
        temps.append(None)
        rains.append(None)
        suns.append(None)
  

# create weather columns
df_merged['temperature'] = temps
df_merged['temperature'] = pd.to_numeric(df_merged['temperature'], errors='coerce')
df_merged['rain'] = rains
df_merged['sun'] = suns

# match density data by state county
densities = []
for row in df_merged.itertuples():
    try:
        densities.append(df_density.loc[df_density['state_county'] == row.state_county]['Density'].values[0])
    except:
        densities.append(None)

# create density column
df_merged['density'] = densities

# remove missing data
df_merged = df_merged.dropna()

# save pre-processed data for easier access
df_merged.to_csv('agg_county_data.csv')

# preview data
df_merged.head()

Our goal for this part of the project was to elucidate the effect of weather on compliance with social distancing measures. We hypothesized that people would be more prone to leaving their houses on day where the weather was nice. We aggregated daily values for precipitation in inches, temperature in Fahrenheit and hours of sunlight for each county in the United States. To evaluate social distancing measures, we used data from Google Mobility Reports which tracks anonymize cellphone movements. Starting February 13th, 2020 we have daily percent change from the baseline at the beginning of the period for trips to retail and recreation centers, parks, transit stations and residential areas. For example we might see that in a certain county, on May 10th, there was a 40% decrease in park visits relative to baseline. By looking at this data we can determine how warmer temperatures and changing weather patterns in the summer months may effect social distancing measures. We believe that this may an additional factor to take into account admist claims of reduced r0 values due to warmer weather. 

# Weather Data EDA

In [ ]:
weather_data = {'rain':df_rain, 'sun': df_sun, 'temp': df_temperature}

def get_weather_df(weather, date):
    data = weather_data[weather]
    df = pd.DataFrame(columns = ['fips', weather])
    df['fips'] = data[data['date']== date].columns[:-1]
    df[weather] = list(data[data['date'] == date].iloc[0,:-1])
    df = df.loc[df[weather] != date]
    df[weather] = df[weather].loc[df[weather] != date].astype(float)
    return df

def plot_weather(weather, date):
    df = get_weather_df(weather, date)
    start = np.min(df[weather].astype(float))
    stop = max(df[weather].astype(float))
    fig = px.choropleth_mapbox(df, geojson=counties, locations='fips', color=weather,
                           color_continuous_scale="Viridis",
                           range_color=(start, stop),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={weather:weather}
                          )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

# plot temperature, rain, and sun
plot_weather('temp', '2020-02-15')
plot_weather('rain', '2020-02-15')
plot_weather('sun', '2020-02-15')


Above, we can see generally warmer temperatures in the Southern regions of the country, Also 6 states are missing data for this date. There was also a large amount of rain in the Pacific Northwest. Also on this date, hours of sunlight seem to have been severely reduced by cloud cover in the Pacific Northwest and parts of the Midwest and South.

# Mobility Data EDA

In [ ]:
indices = ['retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline']

data = pd.read_csv('agg_county_data.csv', parse_dates = ['date'])


In [ ]:
def plot_mobility(data, index, date):
    df = data[data['date'] == date][['fips', index]]
    start = np.min(df[index].astype(float))
    stop = np.max(df[index].astype(float))
    fig = px.choropleth_mapbox(df, geojson=counties, locations='fips', color=index,
                           color_continuous_scale="Viridis",
                           range_color=(start, stop),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={index:index}
                          )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [ ]:
# Look at mobility indices near beginning of time period
for index in indices:
    plot_mobility(data, index, '2020-03-1')

In [ ]:
# Look at mobility indices near end of time period
for index in indices:
    plot_mobility(data, index, '2020-04-10')

Unfortunately google did not have a large degree of coverage for county level data during this time period. Towards the end of the time period studied we have mobility data for around 500 counties. For all the mobility indices besides residential, generally we see a negative change from baseline as people avoid these areas. The change from baseline for residential is positive as people remained at home more under stay-at-home orders.

# Standardize and Add Variables

The weather varibles are standardized to account for regional variation in weather patterns.

In [ ]:
# Standardize weather values in each county
def standardize_df(df, columns):
    scaler = StandardScaler()
    for column in columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')
        fitted = scaler.fit(df[[column]])
        df[[column]] = scaler.transform(df[[column]])
    return df

In [ ]:
data = standardize_df(data, ['rain', 'sun', 'temperature'])
# Add indicators for day of week
data['weekday'] = data['date'].map(lambda x: x.weekday())
# Get number of days since beginning of time period
start = data['date'][0]
data['days'] = data['date'].map(lambda x: (x - start).days)
# One hot encode
data = pd.get_dummies(data, prefix=['weekday'], columns = ['weekday'])
# Add in weekdays for plotting 
data['weekday'] = data['date'].map(lambda x: x.weekday())


We add two new variables: the day of the week and the number of days since the beginning of the time period.

In [ ]:
# Choose predictors and target 
target = 'parks_percent_change_from_baseline'
predictors = ['sun', 'rain', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3',
       'weekday_4', 'weekday_5', 'weekday_6', 'temperature', 'days']

# Linear Regressions

In [ ]:
def fit_OLS(data, predictors, target):
    data = data.dropna()
    X_train, X_test, y_train, y_test = train_test_split(data.loc[:, predictors], 
                                                         data[target], test_size=0.2, 
                                                         random_state = 109)
    X_train =  sm.add_constant(X_train)
    X_test =  sm.add_constant(X_test)
    OLS = sm.OLS(y_train,X_train, missing = 'drop')
    OLS = OLS.fit()
    r2 = sklearn.metrics.r2_score(y_test, OLS.predict(X_test))
    print('Test R2 is: {}'.format(r2))
    return OLS
def plot(data):
    sns.regplot(data['days'], data[target])
    plt.show()
    plt.clf()
    sns.regplot(data['sun'], data[target])
    plt.show()
    plt.clf()
    sns.regplot(data['temperature'], data[target])
    plt.show()
    plt.clf()
    sns.regplot(data['rain'], data[target])
    plt.show()
    plt.clf()
    plt.scatter(data['weekday'], data[target])

In [ ]:
OLS = fit_OLS(data, predictors, target)
OLS.summary()

In [ ]:
plot(data)

We focus on visits to parks as these are the types of nonessential visits that should be avoided in order to mitigate risk and are the most likely to be affected by the weather. In the aggregated data, we see a slight decrease in visits over times. This contrasts to sharp, sudden declines in some of the other metrics (see plot below). While retial and recreation visits dropped precipitously around mid March in resonse to quarantine orders,
visits to parks exhibited a less clear trend. For parks, a 1 standard deviation increase in the hours of sunlight relative to baseline was associated with an average increase in visits to parks of 22% relative to baseline. Interestingly, temperature had a slightly negative effect. Rain had a pronounced effect with larger levels of precipitation leading to fewer visits to parks. The day of the week did not have as pronounced an effect as might be expected as people seemed to go to parks more on Mondays and Tuesdays. 

In [ ]:
plt.scatter(data['days'],data['retail_and_recreation_percent_change_from_baseline'])
plt.show()


In [ ]:
# Fit for state of Massachusetts
mass = data[data['state']== 'Massachusetts']

OLS = fit_OLS(mass, predictors, target)
OLS.summary()

In [ ]:
plot(mass)

We perform the same analysis for the state of Massachussets because it had a large degree of coverage in terms of counties represented in the dataset. On the state level, the effects are more pronounced with a larger $r^2$ value compared to for the entire country. Interestingly, the effect of temperature seems to be reversed and people go to the park more on warmer days. This may be a result of colder average temperatures in this region leading warmer days to fall in the sweet spot for going out. Another interesting result is a pronounced increase in park visits on Saturdays. 

Based on our analysis we conclude that there is a robust relationship between visits to parks and weather. Warmer temperatures in the summer may lead to less compliance in Northern regions and more compliance in Southern regions. Drought conditions, espeically may influence people to seek out parks more. Finally spikes in visits on certain days of the week may indicate the efficacy of stagerring weekends for those who are working at home to prevent large numbers of people from visiting parks at the same time. Next, we will evaluate if we can use this weather data to directly predict Covid-19 case counts.

# Aggregate Predictors for LSTM

In [ ]:
# import saved pre-processed data\
df_merged = pd.read_csv('agg_county_data.csv')

# split into series by county
split = []
for fip in df_merged['fips'].unique():
    split.append(df_merged.loc[df_merged['fips'] == fip])

# seperate predictors from cases to get
# X - a time-series of 10 predictors: temperature, sunshine, rain, 
#   population density and mobility in retail, grocery/pharmacy, parks, transit, work, and residential
# y - a time-series of COVID-19 cases

X = [[(tup.retail_and_recreation_percent_change_from_baseline, tup.grocery_and_pharmacy_percent_change_from_baseline,
   tup.parks_percent_change_from_baseline,tup.transit_stations_percent_change_from_baseline, 
   tup.workplaces_percent_change_from_baseline, tup.residential_percent_change_from_baseline, 
   tup.temperature, tup.rain, tup.sun, tup.density) for tup in x.itertuples()] for x in split]

y = [[tup.cases for tup in x.itertuples()] for x in split]

# pad the data to make each series the same length
for i, county in enumerate(X):
    while len(county) < max([len(x) for x in X]):
        y[i].insert(0,0)
        county.insert(0,(0, 0, 0, 0, 0, 0, 0, 0, 0, 0))

# split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

X_train[0][-5:]

In [ ]:
# function from previous HW to plot loss over epochs
def plot_training_history(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1,len(loss)+1)

    plt.figure()
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('epoch')
    plt.legend()
    plt.show()

# Run GRU and LSTM Models

We attempted to model COVID-19 cases in each county using GRU and LSTM models. We experimented with two different types of models (GRU and LSTM) as well as various types of inputs. 

First we used a GRU model using the cumulative cases for each county.

In [ ]:
# GRU model parameters
hidden_neurons = 100
optimizer = "adam"
loss = "MSE"
batch_size = 8
epochs = 400
verbose = 2

# define GRU model
model_gru = Sequential()
model_gru.add(Input(shape=(68, 10)))
model_gru.add(GRU(hidden_neurons, return_sequences = True))
model_gru.add(TimeDistributed(Dense(1, activation = 'relu')))

# compile GRU model
model_gru.compile(optimizer=optimizer, loss=loss)
model_gru.summary()

In [ ]:
# fit model
history_gru = model_gru.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
                    validation_data=(X_test, y_test), verbose=verbose)

Then we used a GRU model on the log of the number of cases in order to provide the model with a simpler, linear model to predict.

In [ ]:
# GRU model on log cases
hidden_neurons = 100
optimizer = "adam"
loss = "MSE"
batch_size = 8
epochs = 400
verbose = 2

# define GRU model
model_gru_log = Sequential()
model_gru_log.add(Input(shape=(68, 10)))
model_gru_log.add(GRU(hidden_neurons, return_sequences = True))
model_gru_log.add(TimeDistributed(Dense(1, activation = 'relu')))

# compile GRU model
model_gru_log.compile(optimizer=optimizer, loss=loss)
model_gru_log.summary()


In [ ]:
# fit model
history_gru_log = model_gru_log.fit(np.array(X_train), np.log(np.array(y_train) + 1) , batch_size=batch_size, epochs=epochs, 
                    validation_data=(np.array(X_test), np.log(np.array(y_test) + 1)), verbose=verbose)


Next we switched to an LSTM model on the log of the number of cases in each county. 

In [ ]:
# LSTM model parameters
hidden_neurons = 100
optimizer = "adam"
loss = "MSE"
batch_size = 8
epochs = 100
verbose = 2

# define LSTM model
model_lstm = Sequential()
model_lstm.add(Input(shape=(68, 10)))
model_lstm.add(LSTM(hidden_neurons, return_sequences = True))
model_lstm.add(TimeDistributed(Dense(1, activation = 'relu')))

# compile LSTM model
model_lstm.compile(optimizer=optimizer, loss=loss)
model_lstm.summary()

In [ ]:
# fit model
history_lstm = model_lstm.fit(np.array(X_train), np.log(np.array(y_train) + 1) , batch_size=batch_size, epochs=epochs, 
                    validation_data=(np.array(X_test), np.log(np.array(y_test) + 1)), verbose=verbose)


We then ran the same LSTM model on the log of the number of cases but using a much smaller number of hidden neurons. We wanted to test if a smaller LSTM could capture simpler patterns in the data and reduce volatility in our results.

In [ ]:
# LSTM model parameters
hidden_neurons = 10
optimizer = "adam"
loss = "MSE"
batch_size = 8
epochs = 50
verbose = 2

# define LSTM model
model_lstm_mini = Sequential()
model_lstm_mini.add(Input(shape=(68, 10)))
model_lstm_mini.add(LSTM(hidden_neurons, return_sequences = True))
model_lstm_mini.add(TimeDistributed(Dense(1, activation = 'relu')))

# compile LSTM model
model_lstm_mini.compile(optimizer=optimizer, loss=loss)
model_lstm_mini.summary()

In [ ]:
# fit model
history_lstm_mini = model_lstm_mini.fit(np.array(X_train), np.log(np.array(y_train) + 1) , batch_size=batch_size, epochs=epochs, 
                    validation_data=(np.array(X_test), np.log(np.array(y_test) + 1)), verbose=verbose)


Finally, we decided to use the original LSTM architecture, this time attempting to predict the number of new cases rather than the cumulative number of cases for each county. 

In [ ]:
# y vectors based on daily new cases
y_train_new_cases = []
for county in y_train:
    new_cases = [0]
    for i in range(1, len(county)):
        new_cases.append(county[i] - county[i-1])
    y_train_new_cases.append(new_cases)

y_test_new_cases = []
for county in y_test:
    new_cases = [0]
    for i in range(1, len(county)):
        new_cases.append(county[i] - county[i-1])
    y_test_new_cases.append(new_cases)


In [ ]:
# LSTM model parameters
hidden_neurons = 100
optimizer = "adam"
loss = "MSE"
batch_size = 8
epochs = 100
verbose = 2

# define LSTM model
model_lstm_new_cases = Sequential()
model_lstm_new_cases.add(Input(shape=(68, 10)))
model_lstm_new_cases.add(LSTM(hidden_neurons, return_sequences = True))
model_lstm_new_cases.add(TimeDistributed(Dense(1, activation = 'relu')))

# compile LSTM model
model_lstm_new_cases.compile(optimizer=optimizer, loss=loss)
model_lstm_new_cases.summary()


In [ ]:
# fit model
history_lstm_new_cases = model_lstm_new_cases.fit(np.array(X_train), np.array(y_train_new_cases), batch_size=batch_size, epochs=epochs, 
                    validation_data=(np.array(X_test), np.array(y_test_new_cases)), verbose=verbose)


In [ ]:
# Save weights for each model above
model_gru.save_weights('model_gru.h5')
model_gru_log.save_weights('model_gru_log.h5')
model_lstm.save_weights('model_lstm.h5')
model_lstm_mini.save_weights('model_lstm_mini.h5')
model_lstm_new_cases.save_weights('model_lstm_new_cases.h5')

# training accuracy plotted by epoch
plot_training_history(history_gru)
plot_training_history(history_gru_log)
plot_training_history(history_lstm)
plot_training_history(history_lstm_mini)
plot_training_history(history_lstm_new_cases)


In [ ]:
# load saved models
model_gru.load_weights('model_gru.h5')
model_gru_log.load_weights('model_gru_log.h5')
model_lstm.load_weights('model_lstm.h5')
model_lstm_mini.load_weights('model_lstm_mini.h5')
model_lstm_new_cases.load_weights('model_lstm_new_cases.h5')


The LSTM models resulted in much lower losses accross the datasets as compared to the GRU models. We plotted the cumulative predicted and actual number of cases for each county in the test set for each model from above.

In [ ]:
# plot test predictions and label by county name from original GRU model

# traceback each county in test to find county name
fips = []
for row in X_test:
    for tup in df_merged.itertuples():
        if row[-1] == (tup.retail_and_recreation_percent_change_from_baseline, tup.grocery_and_pharmacy_percent_change_from_baseline, 
                   tup.parks_percent_change_from_baseline,tup.transit_stations_percent_change_from_baseline, 
                   tup.workplaces_percent_change_from_baseline, tup.residential_percent_change_from_baseline, tup.temperature, tup.rain, tup.sun, tup.density):
            fips.append(tup.fips)
 
# plot true cases and predicted cases over time for test data
fig, ax = plt.subplots(17, 3, figsize=(15, 80))

predicted_cases = model_gru.predict(X_test)
for i in range(51):
    ax[i // 3][i % 3].plot(np.arange(0, 68) , predicted_cases[i], label = 'Predicted Cases')
    ax[i // 3][i % 3].plot(np.arange(0, 68) , y_test[i], label = 'True Cases')
    ax[i // 3][i % 3].set_title(df_merged.loc[df_merged['fips'] == fips[i]]['state_county'].iloc[0])
    ax[i // 3][i % 3].legend()

plt.show()

In [ ]:
# plot test predictions and label by county name from original GRU model on log of cumulative cases
fig, ax = plt.subplots(17, 3, figsize=(15, 80))

predicted_cases = model_gru_log.predict(X_test)
for i in range(51):
    ax[i // 3][i % 3].plot(np.arange(0, 68) , np.exp(predicted_cases[i]) - 1, label = 'Predicted Cases')
    ax[i // 3][i % 3].plot(np.arange(0, 68) , y_test[i], label = 'True Cases')
    ax[i // 3][i % 3].set_title(df_merged.loc[df_merged['fips'] == fips[i]]['state_county'].iloc[0])
    ax[i // 3][i % 3].legend()

plt.show()


In [ ]:
# plot test predictions and label by county name from original LSTM model on log of cumulative cases

# plot true cases and predicted cases over time for test data
fig, ax = plt.subplots(17, 3, figsize=(15, 80))

predicted_cases = model_lstm.predict(X_test)
for i in range(51):
    ax[i // 3][i % 3].plot(np.arange(0, 68) , np.exp(predicted_cases[i]) - 1, label = 'Predicted Cases')
    ax[i // 3][i % 3].plot(np.arange(0, 68) , y_test[i], label = 'True Cases')
    ax[i // 3][i % 3].set_title(df_merged.loc[df_merged['fips'] == fips[i]]['state_county'].iloc[0])
    ax[i // 3][i % 3].legend()

plt.show()


In [ ]:
# plot test predictions and label by county name from LSTM model wither fewer hidden neurons on log of cumulative cases
fig, ax = plt.subplots(17, 3, figsize=(15, 80))

predicted_cases = model_lstm_mini.predict(X_test)
for i in range(51):
    ax[i // 3][i % 3].plot(np.arange(0, 68) , np.exp(predicted_cases[i]) - 1, label = 'Predicted Cases')
    ax[i // 3][i % 3].plot(np.arange(0, 68) , y_test[i], label = 'True Cases')
    ax[i // 3][i % 3].set_title(df_merged.loc[df_merged['fips'] == fips[i]]['state_county'].iloc[0])
    ax[i // 3][i % 3].legend()

plt.show()

In [ ]:
# plot test predictions and label by county name from LSTM model trained on daily new cases
fig, ax = plt.subplots(17, 3, figsize=(15, 80))

predicted_new_cases = model_lstm_new_cases.predict(X_test)
for i in range(51):
    
    pred_cumulative_cases = predicted_new_cases[i][0]
    for j in range(1, 68):
        pred_cumulative_cases = np.append(pred_cumulative_cases, predicted_new_cases[i][j] + pred_cumulative_cases[j - 1])

    ax[i // 3][i % 3].plot(np.arange(0, 68) , pred_cumulative_cases, label = 'Predicted Cases')
    ax[i // 3][i % 3].plot(np.arange(0, 68) , y_test[i], label = 'True Cases')
    ax[i // 3][i % 3].set_title(df_merged.loc[df_merged['fips'] == fips[i]]['state_county'].iloc[0])
    ax[i // 3][i % 3].legend()

plt.show()

## Discussion of RNN Models

The largest difference in model performance resulted from using the log of cases rather than the total number of cases. This is most likely because the model could more easily predict a linear relationship rather than an exponential relationship given the small sample size of only 500 counties.

Additionally, training a smaller LSTM model resulted in the number of cases increasing in large increments rather than a continuous fashion.

Finally, predicting new cases rather than cumulative cases smoothed out much the model's ouput but also decreased accuracy slightly. It seems this model could more accurately model exponential growth but lacked much of the granularity of the LSTM and GRU models trained on the log number of cases.

# Aggregate Data by County

In addition to the above models, we looked for correlations between the various predictors and R-proxy values for each county. We began by aggregating data by county and deriving R-proxy levels for each county.

In [ ]:
# split cases by county
cases_by_county = df_merged.loc[df_merged['state_county'] == 'Arizona Maricopa'].groupby('date').agg({'cases':'sum'})
cases_by_county = cases_by_county.rename(columns={'cases':'Arizona Maricopa'})

for state_county in df_merged['state_county'].unique():
    if state_county != 'Arizona Maricopa':
        new_county = df_merged.loc[df_merged['state_county'] == state_county].groupby('date').agg({'cases':'sum'})
        new_county = new_county.rename(columns={'cases':state_county})
        cases_by_county = cases_by_county.join(new_county)

cases_by_county = cases_by_county.fillna(0).reset_index().drop('date', axis=1)
cases_by_county.head()


In [ ]:
# find mean 
temp_by_county = df_merged.groupby('state_county').agg({'temperature':'mean'})
rain_by_county = df_merged.groupby('state_county').agg({'rain':'mean'})
sun_by_county = df_merged.groupby('state_county').agg({'sun':'mean'})
density_by_county = df_merged.groupby('state_county').agg({'density':'mean'})
residential_mobility_by_county = df_merged.groupby('state_county').agg({'residential_percent_change_from_baseline':'mean'})
parks_mobility_by_county = df_merged.groupby('state_county').agg({'parks_percent_change_from_baseline':'mean'})
grocery_mobility_by_county = df_merged.groupby('state_county').agg({'grocery_and_pharmacy_percent_change_from_baseline':'mean'})
retail_mobility_by_county = df_merged.groupby('state_county').agg({'retail_and_recreation_percent_change_from_baseline':'mean'})
transit_mobility_by_county = df_merged.groupby('state_county').agg({'transit_stations_percent_change_from_baseline':'mean'})
workplaces_mobility_by_county = df_merged.groupby('state_county').agg({'workplaces_percent_change_from_baseline':'mean'})



In [ ]:
# find r_proxy over a data frame
def r_proxy(start, d, data):
    ans = []
    for t in range(start, len(data)-2*d):
        ans.append((data.iloc[t+2*d]-data.iloc[t+d]) / (data.iloc[t+d]-data.iloc[t]))
    return pd.DataFrame(ans).replace([np.inf, -np.inf], np.nan).fillna(0)

In [ ]:
# find r_proxy by county
r0s_by_county = r_proxy(35, 5, cases_by_county)

# plot bargraph of r_proxy by county
x = np.arange(0, cases_by_county.shape[1] + 10, (cases_by_county.shape[1] + 10) / cases_by_county.shape[1])[:-1]
plt.figure(figsize=(15, 10))
plt.bar(x, r0s_by_county.mean(), width=1)
# plt.xticks(x, labels = r0s_by_county.columns, rotation=90)
plt.ylabel('R0-Proxy')
plt.xlabel('County')
plt.title('R0-Proxy by County')
plt.show()


In [ ]:
# print('Max R0:', r0s_by_county.columns[np.argmax(r0s_by_county.mean())])

# print('Min R0:', r0s_by_county.columns[np.argmin(r0s_by_county.mean())])



print('Max R0:', r0s_by_county.mean()[r0s_by_county.mean() > 3 ])

print('Min R0:', r0s_by_county.mean()[r0s_by_county.mean() < -4 ])


The bar graph above demonstrates the variability in R-proxy values accross the country. Some counties had extremely high R-proxy values (like Richmond Georgia with an R-proxy of about 10.5) while others had much lower averages (like Butler Pennsylvania with an R-proxy of roughly -4.5).

# Regression on R-proxy Values and Predictors



Below we calculated and plotted R-proxy values against the various predictors of the above models.

In [ ]:
# plot R-proxy vs. Temperature
sns.regplot(temp_by_county, r0s_by_county.mean())
plt.ylabel('R0-Proxy')
plt.title('R0-Proxy vs. Temperature')
plt.show()

In [ ]:
# plot R-proxy vs. Rain 
sns.regplot(rain_by_county, r0s_by_county.mean())
plt.ylabel('R0-Proxy')
plt.title('R0-Proxy vs. Rain')
plt.show()

In [ ]:
# plot R-proxy vs. Sun 
sns.regplot(sun_by_county, r0s_by_county.mean())
plt.ylabel('R0-Proxy')
plt.title('R0-Proxy vs. Sun')
plt.show()

In [ ]:
# plot R-proxy vs. Density
sns.regplot(density_by_county.values, r0s_by_county.mean())
plt.ylabel('R0-Proxy')
plt.title('R0-Proxy vs. Density')
plt.show()


In [ ]:
# plot R-proxy vs. Residential Mobility 
sns.regplot(residential_mobility_by_county, r0s_by_county.mean())
plt.ylabel('R0-Proxy')
plt.title('R0-Proxy vs. Residential Mobility')
plt.show()


In [ ]:
# plot R-proxy vs. Retail Mobility
sns.regplot(retail_mobility_by_county, r0s_by_county.mean())
plt.ylabel('R0-Proxy')
plt.title('R0-Proxy vs. Retail Mobility')
plt.show()


In [ ]:
# plot R-proxy vs. Parks Mobility
sns.regplot(parks_mobility_by_county, r0s_by_county.mean())
plt.ylabel('R0-Proxy')
plt.title('R0-Proxy vs. Parks Mobility')
plt.show()


In [ ]:
# plot R-proxy vs. Transit Mobility
sns.regplot(transit_mobility_by_county, r0s_by_county.mean())
plt.ylabel('R0-Proxy')
plt.title('R0-Proxy vs. Transit Mobility')
plt.show()


In [ ]:
# plot R-proxy vs. Grocery Mobility
sns.regplot(grocery_mobility_by_county, r0s_by_county.mean())
plt.ylabel('R0-Proxy')
plt.title('R0-Proxy vs. Grocery Mobility')
plt.show()


In [ ]:
# plot R-proxy vs. Work Mobility
sns.regplot(workplaces_mobility_by_county, r0s_by_county.mean())
plt.ylabel('R0-Proxy')
plt.title('R0-Proxy vs. Work Mobility')
plt.show()


In [ ]:
# Summarize overall regression
exog = ['retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'temperature', 'density', 'sun', 'rain']


OLS = fit_OLS(data, exog, 'cases' )
OLS.summary()



## Discussion of Regressions


The regressions we performed as well as the summary output indicate relatively low correlations between R-proxy and the predictors we used here. The OLS summary directly above actually shows negative coefficients for much of the mobility predictors. Many counties imposed stay-at-home orders when cases and R-proxy began to increase. This would indicate that mobility in these areas decreased with higher R-proxies.

# Sources:

*   Google Mobility Reports https://www.google.com/covid19/mobility/
*   Luo, Wei, et al. “The Role of Absolute Humidity on Transmission Rates of the COVID-19 Outbreak.” 2020, doi:10.1101/2020.02.12.20022467.
* World Weather Online https://www.worldweatheronline.com/ 

